In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import utils
import numpy as np
import spectral.io.envi as envi
from gui.mappers import ByteColorMap, BandMapper

input_file = envi.open('../hypernet-data/f080611t01p00r07rdn_c/f080611t01p00r07rdn_c_sc01_ort_img.hdr')
input_data = input_file[:,:,:]

byte_color_map = ByteColorMap()
byte_color_map.add_point(0, [0, 0, 0])
byte_color_map.add_point(32, [255, 0, 0])
byte_color_map.add_point(128, [0, 255, 0])
byte_color_map.add_point(192, [128, 0, 128])
byte_color_map.add_point(255, [0, 0, 255])

band_mapper = BandMapper(input_data)
output_data = band_mapper.map_colors(byte_color_map, 30)

utils.create_image(output_data)


In [ ]:
import utils
import spectral.io.envi as envi
from gui.mappers import BandMapper

input_file = envi.open('../hypernet-data/f080611t01p00r07rdn_c/f080611t01p00r07rdn_c_sc01_ort_img.hdr')
input_data = input_file[:,:,:]

band_mapper = BandMapper(input_data)
image_data = band_mapper.map_visible(input_file.bands.centers)

normalized_image = utils.normalize_to_byte(image_data)
utils.create_image(normalized_image)


In [ ]:
import utils
import spectral.io.envi as envi
from gui.mappers import BandMapper

input_data = envi.open('../hypernet-data/samson_1.img.hdr')

band_mapper = BandMapper(input_data)
image_data = band_mapper.map_mixed(10, 40, 30)

normalized_image = utils.normalize_to_byte(image_data)
utils.create_image(normalized_image)


In [ ]:
import utils
import spectral.io.envi as envi
import scipy.io
from gui.mappers import BandMapper

mat_file = scipy.io.loadmat('../hypernet-data/PaviaU.mat')
input_data = mat_file['paviaU']

band_mapper = BandMapper(input_data)
image_data = band_mapper.map_single(55)

normalized_image = utils.normalize_to_byte(image_data)
utils.create_image(normalized_image)


In [ ]:
import utils
import scipy.io
from gui.mappers import GroundTruthMapper

mat_file = scipy.io.loadmat('../hypernet-data/PaviaU_gt.mat')
ground_truth_data = mat_file['paviaU_gt']

ground_truth_mapper = GroundTruthMapper(ground_truth_data)
image_data = ground_truth_mapper.map_image()

utils.create_image(image_data)


In [ ]:
import utils
import scipy.io
from gui.mappers import BandMapper

input_mat_file = scipy.io.loadmat('../hypernet-data/PaviaU.mat')
ground_truth_mat_file = scipy.io.loadmat('../hypernet-data/PaviaU_gt.mat')
input_data = input_mat_file['paviaU']
ground_truth_data = ground_truth_mat_file['paviaU_gt']

band_mapper = BandMapper(input_data)
single_input_data = band_mapper.map_single(55)
normalized_input_data = utils.normalize_to_zero_one(single_input_data)
ground_truth_mapper = GroundTruthMapper(ground_truth_data)
mapped_image = ground_truth_mapper.map_image()
image_data = (mapped_image * normalized_input_data).astype(np.uint8)

utils.create_image(image_data)


In [ ]:
import utils
import numpy as np
from gui.mappers import BandMapper

input_data = np.load('../hypernet-data/PaviaU_corrected.npy')
ground_truth = np.load('../hypernet-data/PaviaU_gt.npy')

band_mapper = BandMapper(input_data)
image_data = band_mapper.map_single(55)

normalized_image = utils.normalize_to_byte(image_data)

utils.create_image(normalized_image)


In [ ]:
import utils
import numpy as np
from gui.mappers import BandMapper
from python_research.segmentation import Rect, randomize_positions, extract_rect

input_data = np.load('../hypernet-data/PaviaU_corrected.npy')

bounds = Rect(0, 0, input_data.shape[0], input_data.shape[1])
rects = [
    Rect(0, 0, 100, 100),
    Rect(0, 0, 120, 100),
    Rect(0, 0, 100, 100)
]
rects = list(randomize_positions(rects, bounds))

for rect in rects:
    rect_data = extract_rect(input_data, rect)

    band_mapper = BandMapper(rect_data)
    image_data = band_mapper.map_single(10)

    normalized_image = utils.normalize_to_byte(image_data)
    utils.create_image(normalized_image)

band_mapper = BandMapper(input_data)
image_data = band_mapper.map_single(0)
normalized_image = utils.normalize_to_byte(image_data)
utils.create_image(normalized_image)


In [ ]:
from python_research.preprocessing.attribute_profiles.utils.aux_functions import calculate_pca, normalize_pca, invert_array
from python_research.preprocessing.attribute_profiles.max_tree.max_tree import MaxTree
from python_research.preprocessing.attribute_profiles.generate_ap import generate_ap
from gui.mappers import BandMapper
import time
import numpy as np
import utils

area_thresholds = [100, 500]
stddev_thresholds = [50]
diagonal_thresholds = [25, 50, 100]
moment_thresholds = [0.42]
x = np.load("../hypernet-data/Salinas_corrected.npy")

band_mapper = BandMapper(x)
image_data = band_mapper.map_single(10)
normalized_image = utils.normalize_to_byte(image_data)
utils.create_image(normalized_image)

min_ = np.amin(x)
max_ = np.amax(x)

pca = calculate_pca(x, 5)
pca = normalize_pca(pca, min_, max_)

area_eap = []
stddev_eap = []
diagonal_eap = []
moment_eap = []
start = time.time()
for index in range(0, pca.shape[-1]):
    pc = pca[:, :, index]
    tree_thick = MaxTree(pc, ['area'])
    tree_thin = MaxTree(invert_array(pc), ['area'])
    area_ap = generate_ap(pc, tree_thick, tree_thin, 'area', area_thresholds)
    area_eap.append(area_ap)

print("Salinas moment time: {}".format(time.time() - start))
salinas = np.empty(x.shape)
area_eap = np.array(area_eap)
np_index = 0
for index, eap in enumerate(area_eap):
    for ap in range(0, eap.shape[-1]):
        salinas[:, :, np_index] = eap[:, :, ap]
        np_index += 1
